In [92]:
import torch
import napari
import numpy as np
import os
from src.datasets import create_datasets
from src.unet import UNet

In [ ]:
from torchsummary import summary

model = UNet(n_channels=64, n_classes=1)
# Load the checkpoint
checkpoint = torch.load("checkpoints/checkpoint_epoch15.pth")
# Get the learning rate and remove it from the checkpoint
lr = checkpoint.pop('learning_rate')
# Load the state dictionary into the model
model.load_state_dict(checkpoint)
# Move the model to the device

In [19]:
data = np.load('data_with_centers/0.npz')
bio = data['biosensor'].astype(np.float32)
mask = data['mask']

In [22]:
viewer, image_layer = napari.imshow(mask, name='biosensor')
new_layer = viewer.add_image(bio)

In [98]:
data_path = 'data_with_centers/'
train_percent = 1
bio_len = 64
mask_size = 80

dataset = create_datasets(data_path, train_percent, bool, biosensor_length=bio_len, mask_size=mask_size, augment=False)

In [113]:
data = dataset[0][8]
viewer, image_layer = napari.imshow(data[0].numpy(), name='biosensor')
viewer.add_labels(data[1].numpy(), name='mask')
predictions = model(data[0].unsqueeze(0))
binary_predictions = (predictions.detach().numpy() > 0.5).astype(bool)
viewer.add_labels(binary_predictions, name='predictions')

<Labels layer 'predictions' at 0x24dbb35c460>

In [87]:
data = dataset[0][16]
viewer.add_image(data[0].numpy(), name='biosensor')
viewer.add_labels(data[1].numpy(), name='mask')

<Labels layer 'mask [1]' at 0x24d7dba8070>